# 数据预处理（非救援优先级指标计算）

In [1]:
import pandas as pd
import numpy as np

# 时间窗生成函数（秒为单位）
def generate_time_windows(df, start_Time=0, end_Time=1440, min_window=5, max_window=60):
    """
    为需求点生成服务时间窗（秒为单位）
    
    参数:
    df: 输入的DataFrame
    start_Time: 起始时间范围（24小时制）
    end_Time: 结束时间范围（24小时制）
    min_window: 最小时间窗长度（分钟）
    max_window: 最大时间窗长度（分钟）
    """
    # 将分钟转换为秒
    min_window_seconds = min_window
    max_window_seconds = max_window
    
    # 计算起始时间范围（秒）
    start_min = start_Time  
    start_max = end_Time    
    
    # 生成随机起始时间（秒）
    start_times = np.random.randint(start_min, start_max, len(df))
    
    # 生成随机服务时长（秒）
    durations = np.random.randint(min_window_seconds, max_window_seconds + 1, len(df))
    
    # 计算结束时间（秒）
    end_times = start_times + durations
    
    # 存储为整数秒值
    df['startTime'] = start_times
    df['endTime'] = end_times
    
    return df

# 数据预处理流程
# 1. 读取Excel文件
df = pd.read_excel('Image_Summary.xlsx', sheet_name='image_summary')

# 2. 筛选需要的列
selected_columns = [
    'latitude', 'longitude', 'Total_Damage_Building_area', 'Total_Building_area',
    'Building-No-Damage_area', 'Road-Clear_area', 'Road-Blocked_area'
]
df = df[selected_columns]

# 3. 过滤非零行
filter_condition = (
    (df['Total_Damage_Building_area'] != 0) |
    (df['Building-No-Damage_area'] != 0) |
    (df['Road-Clear_area'] != 0) |
    (df['Road-Blocked_area'] != 0)
)

df_filter = df[filter_condition].reset_index(drop=True)
# 房屋损害程度标定
df_filter = df_filter[df_filter['Total_Building_area'] != 0]
df_filter['Building_Damage_degree'] = df_filter['Total_Damage_Building_area'] / df_filter['Total_Building_area']
# 道路受损程度标定
df_filter['Road_Damage_degree'] = df_filter['Road-Blocked_area'] / (df_filter['Road-Clear_area'] + df_filter['Road-Blocked_area'])
df_filter['Road_Damage_degree'].fillna(0,inplace=True)  # 无道路标识道路非受损
# 人口估算
df_filter['popul_number'] = df_filter['Total_Building_area'] * 0.0005  # 每100m×100m区的人口
# 受灾人数估算
df_filter['victim_number'] = df_filter['popul_number'] * df_filter['Building_Damage_degree']  # 每100m×100m区的人口受灾率
# 需求量估算
omiga, demand_rate = 0.3, 0.1   # 受灾人员物资需求权重、需求量（kg）/人
df_filter["demand"] = (omiga * df_filter['popul_number'] + (1 - omiga) * df_filter['victim_number']) * demand_rate
# 生成时间窗（秒为单位）
df_filter = generate_time_windows(
    df_filter, 
    start_Time=0, 
    end_Time=1380, 
    min_window=5, 
    max_window=60
)

df_filter = df_filter.iloc[:,[0,1,7,8,9,10,11,12,13]]
df_filter

,latitude,longitude,Building_Damage_degree,Road_Damage_degree,popul_number,victim_number,demand,startTime,endTime
0,29.940088,-85.406021,1.000000,0.0,362.8840,362.8840,36.288400,166,180
1,29.939969,-85.406125,0.970277,0.0,292.3125,283.6240,28.623055,325,354
2,29.939882,-85.405887,0.978467,0.0,285.8180,279.6635,28.150985,587,636
3,29.939995,-85.405787,0.990771,0.0,277.9160,275.3510,27.612050,640,667
4,29.937401,-85.401884,1.000000,0.0,266.2275,266.2275,26.622750,1201,1250
...,...,...,...,...,...,...,...,...,...
3598,29.942646,-85.404762,1.000000,0.0,1.1995,1.1995,0.119950,268,301
3599,29.944900,-85.415686,1.000000,0.0,1.0560,1.0560,0.105600,659,670
3600,29.942805,-85.401545,1.000000,0.0,1.0495,1.0495,0.104950,1276,1329
3601,29.950900,-85.428561,1.000000,0.0,0.0305,0.0305,0.003050,683,730


# 救援优先级指标计算

In [2]:
df = df_filter.iloc[:,2:6]  # 筛选救援优先级指标计算的有效列

## 数据去量纲化

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

# 数据标准化函数
def normalize_with_direction(data, cost_indices):
    """
    基于指标方向的数据标准化
    :param data: NumPy数组 (n_samples, n_features)
    :param cost_indices: 负向指标索引列表
    :return: 标准化数据
    """
    normalized = data.copy().astype(float)  # 确保为浮点型
    
    # 处理每个指标
    for col_idx in range(data.shape[1]):
        col_data = data[:, col_idx]
        
        epsilon = 1e-6
        col_min = np.min(col_data) - epsilon
        col_max = np.max(col_data) + epsilon
        
        if col_idx in cost_indices:  # 负向指标（成本型指标）
            normalized[:, col_idx] = (col_max - col_data) / (col_max - col_min)
        else:  # 正向指标（效益型指标）
            normalized[:, col_idx] = (col_data - col_min) / (col_max - col_min)
    
    return normalized

# 数据列赋名
criteria_names = ['C1', 'C2', 'C3', 'C4']
data_array = df.iloc[:, :].to_numpy() 

# 归一化（行-样本；列-属性）
after_data = normalize_with_direction(data_array, [0,1,3])
after_data

array([[9.99998000e-07, 9.99999000e-01, 9.99999997e-01, 2.75570152e-09],
       [2.97242675e-02, 9.99999000e-01, 8.05509660e-01, 2.18416906e-01],
       [2.15338905e-02, 9.99999000e-01, 7.87611252e-01, 2.29330862e-01],
       ...,
       [9.99998000e-07, 9.99999000e-01, 2.80829865e-03, 9.97107888e-01],
       [9.99998000e-07, 9.99999000e-01, 2.75593316e-09, 9.99915948e-01],
       [9.99999000e-01, 9.99999000e-01, 8.83001261e-03, 9.99999997e-01]])

## Entropy指标

In [4]:
def entropy_weight(matrix):
    # 计算比重
    y = matrix / np.sum(matrix, axis=0)

    # 计算熵值
    k = 1 / np.log(matrix.shape[0])
    e = -k * np.sum(y * np.log(y + 1e-10), axis=0)  # 避免log(0)

    # 计算权重
    v = 1 - e
    weights = v / np.sum(v)
    return weights

entropy_weights = entropy_weight(after_data)
print("熵权法权重:", dict(zip(criteria_names, np.round(entropy_weights, 8))))

熵权法权重: {'C1': 0.70390744, 'C2': 0.06726556, 'C3': 0.22346946, 'C4': 0.00535753}


## CRITIC指标

In [5]:
def critic_weight(matrix):
    # 计算标准差
    std = np.std(matrix, axis=0, ddof=1)
    
    # 计算相关系数和冲突性
    n_criteria = matrix.shape[1]
    correlation = np.zeros((n_criteria, n_criteria))
    for i in range(n_criteria):
        for j in range(n_criteria):
            correlation[i, j], _ = pearsonr(matrix[:, i], matrix[:, j])
    
    conflict = np.sum(1 - np.abs(correlation), axis=1)
    
    # 计算信息量
    information = std * conflict
    
    # 计算权重
    weights = information / np.sum(information)
    return weights

critic_weights = critic_weight(after_data)
print("CRITIC权重:", dict(zip(criteria_names, np.round(critic_weights, 8))))

CRITIC权重: {'C1': 0.39556479, 'C2': 0.43392476, 'C3': 0.08680439, 'C4': 0.08370606}


## 复合权重计算（Entropy  and  CRITIC）

In [6]:
def combined_weight(w_entropy, w_critic):
    # 乘法合成
    combined = w_entropy * w_critic
    # 归一化
    normalized = combined / np.sum(combined)
    return normalized

final_weights = combined_weight(entropy_weights, critic_weights)
print("组合权重:", dict(zip(criteria_names, np.round(final_weights, 8))))

组合权重: {'C1': 0.8502644, 'C2': 0.08913084, 'C3': 0.05923531, 'C4': 0.00136944}


## VIKOR排序

In [9]:
def vikor(matrix, weights, alpha=0.5):
    # 确定理想解
    mask = np.ones(matrix.shape[1], dtype=bool)
    
    ideal_best = np.where(mask, np.max(matrix, axis=0), np.min(matrix, axis=0))
    ideal_worst = np.where(mask, np.min(matrix, axis=0), np.max(matrix, axis=0))
    
    # 计算S和R值
    S = np.sum(weights * (ideal_best - matrix) / (ideal_best - ideal_worst + 1e-10), axis=1)
    R = np.max(weights * (ideal_best - matrix) / (ideal_best - ideal_worst + 1e-10), axis=1)
    
    # 计算Q值
    S_min, S_max = np.min(S), np.max(S)
    R_min, R_max = np.min(R), np.max(R)
    Q = alpha * (S - S_min)/(S_max - S_min) + (1 - alpha) * (R - R_min)/(R_max - R_min)
    
    # 生成排序
    ranking = np.argsort(Q)
    return Q, ranking

Q_values, ranking = vikor(after_data, final_weights)

## 数据保存

In [10]:
df_filter['Prority'] = Q_values.tolist()   # 添加优先级数据列
# df_filter = df_filter.loc[:,['latitude','longitude','demand','startTime','endTime','Prority']]
df_filter.to_excel(r'./fin_data.xlsx')